In [1]:
import sys
import os
import json
from datetime import datetime
from pathlib import Path
import logging
import random

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn_extra.cluster import KMedoids

# Chemins de base
BASE_PATH = Path().resolve()  # dossier où se trouve le notebook

# Pour pouvoir importer config.py (à la racine du projet) et utils/
# Adapter selon ta structure exacte :
PROJECT_ROOT = BASE_PATH.parent  # si notebook dans from_similarity_to_clustering/
sys.path.append(str(PROJECT_ROOT))           # pour `from config import *` (cosine_idf/config.py)
sys.path.append(str(BASE_PATH))             # pour `from utils import ...` (from_similarity_to_clustering/utils)

from config import *  # importe NOM_ANALYSE, AUTEURS, etc. depuis ton config global
from utils import (
    explo_clustering,
    plot_similarity_distribution_from_mat,
    evaluate_kmedoids_clustering,
    viz_clustering
)

print("BASE_PATH =", BASE_PATH)
print("PROJECT_ROOT =", PROJECT_ROOT)


/Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering
BASE_PATH = /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering
PROJECT_ROOT = /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf


In [6]:
# === Paramètres d'analyse (overrides possibles de ceux importés depuis config.py) ===

# Métadonnées
NOM_ANALYSE = "clustering_termes_HPO"
AUTEURS = "Axel"
OBJECTIF = "Clustering après calcul de similarité entre termes HPO pour faire du clustering."
ETAPES = "similarité → distance → standardisation → PCA → KMedoids → visualisation"

# Paramètres globaux de l'analyse
INPUT_IS_BINARY_MATRIX = False       # True si matrix_for_clustering est binaire (tu sautes la transfo en distance)
PLOTTING_SIM_DIST = True
DIST_TRANSFO = "1 - sim"            # purement informatif ici
STAND_TRANSFO = True                # standardisation des distances
EXECUTE_PCA = False
N_COMPONENTS_PCA = 50
STOP_AFTER_METRICS = False          # True pour ne faire que les métriques KMedoids (pas la suite)
KMEDOIDS_MAX_K = 15                 # max k pour la recherche de métriques
N_CLUSTERS = 8                      # k final utilisé pour KMedoids
MIN_FREQ = 0.05                     # paramètres pour l’enrichissement
P_THRESHOLD = 0.05

# Fichiers d'entrée
EHR_HPO_PATH = os.path.join(base_path, 'data', 'subset500_simulated_patients_SHEPHERD_updated_2025hpo.csv')
SIM_MAT_PATH = os.path.join(base_path, 'data', 'cosine_similarity_wIDF_SHEPHERD.csv') # A CHANGER SURTOUT DANS LE MAIN
HPO_ONTOLOGY_PATH = os.path.join(base_path, 'data', 'hpoterms_list_v2025_01_16.txt')

# Fichiers de sortie
OUTPUT_FOLDER = os.path.join(base_path, 'output')

print("EHR_HPO_PATH      =", EHR_HPO_PATH)
print("SIM_MAT_PATH      =", SIM_MAT_PATH)
print("HPO_ONTOLOGY_PATH =", HPO_ONTOLOGY_PATH)
print("OUTPUT_FOLDER     =", OUTPUT_FOLDER)


EHR_HPO_PATH      = /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/data/subset500_simulated_patients_SHEPHERD_updated_2025hpo.csv
SIM_MAT_PATH      = /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/data/cosine_similarity_wIDF_SHEPHERD.csv
HPO_ONTOLOGY_PATH = /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/data/hpoterms_list_v2025_01_16.txt
OUTPUT_FOLDER     = /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output


In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_filename = os.path.join(OUTPUT_FOLDER, f"clustering_analysis_{timestamp}.log")

# Configuration du logger global
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(log_filename),
        logging.StreamHandler()
    ]
)

2026-01-21 16:19:35,474 - INFO - Initialisation du notebook terminée.


In [8]:
logging.info("Chargement des données")

# Matrice de similarité
sim_df = pd.read_csv(SIM_MAT_PATH, index_col=0)

# EHR x HPO (binaire ou pondéré)
ehr_hpo = pd.read_csv(EHR_HPO_PATH, index_col=0)

# Alignement des patients avec la matrice de similarité
ehr_hpo_subset = ehr_hpo.loc[sim_df.index]

logging.info("Étape de filtration : retrait CP (HP:0100021)")
logging.info(f"Dimension avant filtration : {ehr_hpo_subset.shape}")

# Retrait du terme CP dans la matrice EHR-HPO
if "HP:0100021" in ehr_hpo_subset.columns:
    ehr_hpo_subset = ehr_hpo_subset.drop(columns=["HP:0100021"])
    print("ehr_hpo_subset sans HP:0100021 :", ehr_hpo_subset.shape)

# Retrait du terme CP dans la matrice de similarité, si présent
if "HP:0100021" in sim_df.columns:
    sim_df = sim_df.drop(columns=["HP:0100021"])
    print("sim_df sans HP:0100021 :", sim_df.shape)

logging.info(f"Dimension après filtration : {ehr_hpo_subset.shape}")
logging.info(f"Dimension de la matrice de similarité : {sim_df.shape}")

# Chargement ontologie (table tabulée déjà préparée)
hpoterms = pd.read_csv(HPO_ONTOLOGY_PATH, sep="\t", header=0)

# Vérification de cohérence
if sim_df.shape[0] != ehr_hpo_subset.shape[0]:
    raise ValueError("Mismatch entre sim_df et ehr_hpo_subset : même nombre de patients attendu.")

logging.info("Données chargées et alignées avec succès.")


2026-01-21 16:18:49,010 - INFO - Chargement des données


2026-01-21 16:18:50,300 - INFO - Étape de filtration : retrait CP (HP:0100021)
2026-01-21 16:18:50,303 - INFO - Dimension avant filtration : (500, 6334)
2026-01-21 16:18:50,418 - INFO - Dimension après filtration : (500, 6333)
2026-01-21 16:18:50,420 - INFO - Dimension de la matrice de similarité : (500, 500)
2026-01-21 16:18:50,569 - INFO - Données chargées et alignées avec succès.


ehr_hpo_subset sans HP:0100021 : (500, 6333)


In [ ]:
logging.info("Prétraitement de la matrice pour clustering")

# 1. Distribution des similarités
if INPUT_IS_BINARY_MATRIX:
    logging.info("Données binaires détectées : pas de transfo en distance ni standardisation.")
    matrix_for_clustering = sim_df.values

else:
    if PLOTTING_SIM_DIST:
        logging.info("Étape 1 : Distribution des similarités")
        plot_similarity_distribution_from_mat.plot_similarity_distribution_from_matrix(
            sim_df,
            OUTPUT_FOLDER=f"{OUTPUT_FOLDER}/plot_similarity_distribution.png",
            title="Similarity Distribution"
        )

    # 2. Transformation en distance
    logging.info("Étape 2 : Transformation des similarités en distances (1 - sim)")
    dist_df = 1 - sim_df

    # 3. Standardisation
    if STAND_TRANSFO:
        logging.info("Étape 3 : Standardisation de la matrice de distance")
        mean_distance = np.mean(dist_df)
        std_distance = np.std(dist_df)

        standardized_dist_matrix = (dist_df - mean_distance) / std_distance

        # Remplacement des valeurs infinies
        standardized_dist_matrix = np.where(np.isinf(standardized_dist_matrix), 1e10, standardized_dist_matrix)

        # Décalage pour distances positives
        min_value = np.min(standardized_dist_matrix)
        if min_value < 0:
            shifted_dist_matrix = standardized_dist_matrix + abs(min_value)
        else:
            shifted_dist_matrix = standardized_dist_matrix

        # Diagonale à 0
        np.fill_diagonal(shifted_dist_matrix, 0)

    else:
        shifted_dist_matrix = dist_df.values

    matrix_for_clustering = shifted_dist_matrix

logging.info("Prétraitement terminé. matrix_for_clustering.shape = %s", matrix_for_clustering.shape)


2026-01-21 16:20:40,368 - INFO - Prétraitement de la matrice pour clustering
2026-01-21 16:20:40,371 - INFO - Étape 1 : Distribution des similarités
2026-01-21 16:20:41,150 - INFO - Étape 2 : Transformation des similarités en distances (1 - sim)
2026-01-21 16:20:41,159 - INFO - Étape 3 : Standardisation de la matrice de distance
/Users/axel/miniconda3/envs/cluster_analysis/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)
2026-01-21 16:20:41,191 - INFO - Prétraitement terminé. matrix_for_clustering.shape = (500, 500)


✅ Distribution saved to: /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/plot_similarity_distribution.png


### PCA (optionnelle)

In [13]:
if EXECUTE_PCA:
    logging.info(f"Étape 4 : PCA avec {N_COMPONENTS_PCA} composantes")
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(matrix_for_clustering)

    pca = PCA(n_components=N_COMPONENTS_PCA, random_state=123)
    pca_result = pca.fit_transform(scaled_data)

    logging.info("PCA terminée. pca_result.shape = %s", pca_result.shape)
else:
    logging.info("PCA non exécutée.")
    pca_result = None


2026-01-21 16:22:10,086 - INFO - PCA non exécutée.


### Métriques KMedoids (silhouette, DBI, WCSS)

In [15]:
logging.info(f"Étape 5 : Évaluation KMedoids jusqu'à {KMEDOIDS_MAX_K} clusters")

if EXECUTE_PCA:
    metric = "euclidean"
    data_for_metrics = pca_result
else:
    metric = "precomputed"
    data_for_metrics = matrix_for_clustering

silhouette_scores, dbi_scores, inertia_scores = evaluate_kmedoids_clustering.evaluate_kmedoids_clustering(
    data_for_metrics,
    KMEDOIDS_MAX_K,
    metric
)

ks = list(range(2, KMEDOIDS_MAX_K + 1))

# Silhouette
evaluate_kmedoids_clustering.plot_metric(
    x_values=ks,
    y_values=silhouette_scores,
    title="Silhouette Score - KMedoids",
    y_label="Silhouette Score",
    file_path_base=f"{OUTPUT_FOLDER}/silhouette_score",
    color="green"
)

# Davies-Bouldin Index
evaluate_kmedoids_clustering.plot_metric(
    x_values=ks,
    y_values=dbi_scores,
    title="Davies-Bouldin Index - KMedoids",
    y_label="Davies-Bouldin Index",
    file_path_base=f"{OUTPUT_FOLDER}/davies_bouldin_index",
    color="red"
)

# WCSS (Inertia)
evaluate_kmedoids_clustering.plot_metric(
    x_values=ks,
    y_values=inertia_scores,
    title="Elbow Method - KMedoids",
    y_label="WCSS (Inertia)",
    file_path_base=f"{OUTPUT_FOLDER}/elbow_method",
    color="blue"
)

logging.info("Métriques KMedoids calculées et sauvegardées.")

if STOP_AFTER_METRICS:
    logging.info("Arrêt après calcul des métriques (STOP_AFTER_METRICS=True).")


2026-01-21 16:22:58,084 - INFO - Étape 5 : Évaluation KMedoids jusqu'à 15 clusters


2026-01-21 16:23:03,411 - INFO - Métriques KMedoids calculées et sauvegardées.


### Kmedoids final & visualisations (UMAP, t-SNEE, PCA)

In [ ]:
if not STOP_AFTER_METRICS:
    logging.info(f"Étape 6 : KMedoids final avec {N_CLUSTERS} clusters")

    determined_n_clusters = N_CLUSTERS
    suffix = "euclidean" if EXECUTE_PCA else "cosine"

    if EXECUTE_PCA:
        kmedoid = KMedoids(n_clusters=determined_n_clusters, metric="euclidean", random_state=123)
        kmedoid_result = kmedoid.fit(pca_result)
        target_df = pca_result
    else:
        kmedoid = KMedoids(n_clusters=determined_n_clusters, metric="precomputed", random_state=123)
        kmedoid_result = kmedoid.fit(matrix_for_clustering)
        target_df = matrix_for_clustering

    labels = kmedoid_result.labels_

    logging.info("Étape 7 : Visualisation des clusters (UMAP, t-SNE, PCA)")

    viz_clustering.apply_umap(target_df, labels, f"{OUTPUT_FOLDER}/umap_2D_{suffix}")
    viz_clustering.apply_tsne(target_df, labels, f"{OUTPUT_FOLDER}/tsne_2D_{suffix}")
    viz_clustering.apply_pca(target_df, labels, f"{OUTPUT_FOLDER}/pca_2D_{suffix}")

    logging.info("Visualisations de clustering sauvegardées.")

2026-01-21 16:26:14,751 - INFO - Étape 6 : KMedoids final avec 8 clusters
2026-01-21 16:26:14,808 - INFO - Étape 7 : Visualisation des clusters (UMAP, t-SNE, PCA)
/Users/axel/miniconda3/envs/cluster_analysis/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/Users/axel/miniconda3/envs/cluster_analysis/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

/Users/axel/miniconda3/envs/cluster_analysis/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/Users/axel/miniconda3/envs/cluster_analysis/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

/Users/axel/miniconda

### Résumés des clusters et tables globales

In [ ]:
print(matrix_for_clustering.shape)
print(len(ehr_hpo_subset.index))

(500, 500)


In [ ]:
if not STOP_AFTER_METRICS:
    logging.info("Étape 8 : Génération du résumé des clusters")

    df_for_clustering = pd.DataFrame(
        matrix_for_clustering,
        index=ehr_hpo_subset.index,
        columns=ehr_hpo_subset.index
    )

    global_c_sum = explo_clustering.cluster_size_summary(
        labels,
        ehr_hpo_subset,
        df_for_clustering,
        output_path=f"{OUTPUT_FOLDER}/cluster_summary.csv"
    )

    # Clusters non vides
    non_empty_clusters = global_c_sum[global_c_sum["Nb EHR in Cluster"] > 0]["Cluster"].tolist()
    print("Clusters non vides :", non_empty_clusters)

    # Pie chart
    logging.info("Étape 9 : Pie chart des tailles de clusters")
    explo_clustering.plot_cluster_pie_chart(
        cluster_table=global_c_sum,
        output_folder=f"{OUTPUT_FOLDER}",
        n_clusters=determined_n_clusters,
        title="pie_chart"
    )

    # Cluster-level summary
    logging.info("Étape 10 : Analyse des caractéristiques par cluster")
    c_level_sum = explo_clustering.cluster_summary_table_global_approach(
        labels,
        ehr_hpo_subset,
        hpoterms,
        P_THRESHOLD,
        MIN_FREQ
    )

    # EHR level summary table
    logging.info("Étape 11 : Table de résumé par patient")
    explo_clustering.ehr_level_summary_table_with_enrichment(
        labels,
        ehr_hpo_subset,
        c_level_sum,
        output_path=f"{OUTPUT_FOLDER}/ehr_level_summary_table.csv"
    )

2026-01-21 16:28:30,647 - INFO - Étape 8 : Génération du résumé des clusters
2026-01-21 16:28:36,130 - INFO - Étape 9 : Pie chart des tailles de clusters


Résumé sauvegardé sous : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/cluster_summary.csv
Clusters non vides : [0, 1, 2, 3, 4, 5, 6, 7]


2026-01-21 16:28:37,610 - INFO - Étape 10 : Analyse des caractéristiques par cluster


[INFO] Graphique sauvegardé : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/camembert_8.html


2026-01-21 16:28:39,985 - INFO - Étape 11 : Table de résumé par patient


Résumé EHR sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/ehr_level_summary_table.csv


### Analyse détaillée par cluster

In [26]:
if not STOP_AFTER_METRICS:
    logging.info("Étape 12 : Analyse détaillée par cluster")

    for cluster_id in non_empty_clusters:
        logging.info(f"Traitement du cluster {cluster_id}")

        cluster_folder = f"{OUTPUT_FOLDER}/cluster_{cluster_id}"
        os.makedirs(cluster_folder, exist_ok=True)

        explo_clustering.format_summary_table_proportion(
            c_level_sum,
            cluster_id,
            output_path=f"{cluster_folder}/formatted_table.csv"
        )

        explo_clustering.volcano_plot_cluster_enrichment(
            c_level_sum,
            cluster_id,
            output_path=f"{cluster_folder}/enrichment"
        )

        try:
            explo_clustering.compare_cluster_vs_noncluster_plot(
                c_level_sum,
                cluster_id,
                output_path=f"{cluster_folder}/prop"
            )
        except Exception as e:
            logging.error(f"Erreur lors du plot du cluster {cluster_id} : {e}")

    logging.info("Analyse complète terminée.")


2026-01-21 16:28:53,420 - INFO - Étape 12 : Analyse détaillée par cluster
2026-01-21 16:28:53,457 - INFO - Traitement du cluster 0


Tableau formaté sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/cluster_0/formatted_table.csv
Volcano plot interactif sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/cluster_0/enrichment.html
Volcano plot PNG sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/cluster_0/enrichment.png
Scatter plot interactif sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/cluster_0/prop.html


2026-01-21 16:28:57,565 - INFO - Traitement du cluster 1


Scatter plot PNG sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/cluster_0/prop.png
Tableau formaté sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/cluster_1/formatted_table.csv
Volcano plot interactif sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/cluster_1/enrichment.html
Volcano plot PNG sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/cluster_1/enrichment.png
Scatter plot interactif sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/Stri

2026-01-21 16:28:58,439 - INFO - Traitement du cluster 2


...

Scatter plot interactif sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/cluster_5/prop.html
Scatter plot PNG sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/cluster_5/prop.png
Tableau formaté sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/cluster_6/formatted_table.csv
Volcano plot interactif sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/cluster_6/enrichment.html
Volcano plot PNG sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMat

2026-01-21 16:29:01,943 - INFO - Traitement du cluster 7


Scatter plot interactif sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/cluster_6/prop.html
Scatter plot PNG sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/cluster_6/prop.png
Tableau formaté sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/cluster_7/formatted_table.csv
Volcano plot interactif sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/cluster_7/enrichment.html
Volcano plot PNG sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMat

2026-01-21 16:29:02,411 - INFO - Analyse complète terminée.


Scatter plot PNG sauvegardé dans : /Users/axel/Documents/Axel/stages/IaDiag_stageM2/docs_transferes/Moussa/StringMatching_14042022/ext/clustering/cosine_idf/from_similarity_to_clustering/output/cluster_7/prop.png


### Cluster with cat

In [28]:
viz_clustering.apply_umap(X, labels, output_path, color_values=color_var, color_name="variant_presence")
viz_clustering.apply_tsne(X, labels, output_path, color_values=color_var, color_name="variant_presence")
viz_clustering.apply_pca(X, labels, output_path, color_values=color_var, color_name="variant_presence")

NameError: name 'X' is not defined

### Idées : 

Il serait intéressant d’explorer l’interprétabilité des clusters en se focalisant non plus sur les termes HPO individuels, mais sur leurs grandes branches physiopathologiques. L’idée serait de récupérer, pour chaque patient, les grands systèmes représentés dans ses phénotypes (par exemple système nerveux, cardiovasculaire, endocrinien, etc.), puis d’agréger ces informations au niveau des clusters afin d’obtenir une image plus synthétique de leur composition clinique. Concrètement, une première étape consiste à mapper chaque terme HPO aux grandes catégories top-level définies dans l’ontologie. On construit ensuite une matrice patient × branches où chaque cellule quantifie l’importance de la branche pour ce patient. Cette importance peut être définie de plusieurs manières, et plusieurs normalisations complémentaires sont pertinentes. Une normalisation structurelle permet de corriger le fait que certaines branches contiennent beaucoup plus de termes que d’autres ; une normalisation centrée patient permet de capturer la distribution relative du phénotype (et non le volume absolu) ; et une normalisation centrée population permet d’intégrer la rareté ou la spécificité d’une branche au sein de la cohorte, notamment utile pour mettre en évidence des signatures différentielles entre clusters. Ces normalisations peuvent se combiner, car elles corrigent des biais distincts : la taille de l’ontologie, la densité du phénotype individuel et la fréquence des phénotypes dans la population. La combinaison la plus informative pour la visualisation qualitative semble être la correction par la taille de branche suivie de la normalisation intra-patient, car elle produit véritablement un « profil phénotypique » des individus indépendamment du volume de leur annotation. Pour comparer ensuite les clusters entre eux, il est possible d’appliquer une normalisation populationnelle supplémentaire (sous forme de Z-score ou de TF-IDF) afin de faire ressortir les branches les plus caractéristiques. Au niveau de la visualisation, on pourra représenter les résultats sous forme de heatmaps patient × branches (regroupés par cluster), puis compléter par des profils moyens au niveau du cluster, voire des visualisations polaires donnant une signature rapide du cluster. Ce module fournirait un moyen puissant d’interpréter les clusters en termes de grands systèmes physiopathologiques, ce qui est généralement plus intelligible et exploitable que des listes de termes HPO bruts.

Enrichissement fonctionnel par cluster
 